In [64]:
import joblib
import pandas as pd

In [65]:
inputs1 = {'Action':2, 'Adventure':5,
       'Animation':2, 'Comedy':3, 'Crime':4, 'Documentary':4, 'Drama':4,
       'Fantasy':1, 'Film-Noir':1, 'Horror':2, 'Musical':0, 'Mystery':5, 'Romance':4,
       'Sci-Fi':2, 'Thriller':1, 'War':5, 'Western':5, '(no genres listed)':1}

In [66]:
inputs2 = {'Action':5/19, 'Adventure':5/19, 'Animation':5/19, 'Comedy':5/19, 'Crime':5/19,
       'Documentary':5/19, 'Drama':5/19, 'Fantasy':5/19, 'Film-Noir':5/19, 'Horror':5/19,
       'Musical':5/19, 'Mystery':5/19, 'Romance':5/19, 'Sci-Fi':5/19, 'Thriller':5/19,
       'War':5/19, 'Western':5/19, '(no genres listed)':5/19}


In [67]:
#Load both models and have them make predictions
model1 = joblib.load("Models/weights_model.pkl")
model2 = joblib.load("Models/counts_model.pkl")

prediction1 = model1.predict(pd.DataFrame(inputs1, index = [0]))
prediction2 = model2.predict(pd.DataFrame(inputs2, index = [0]))

In [68]:
#This combines the predictions to make an overall cluster
cluster = int(str(prediction1[0])+str(prediction2[0]))

In [69]:
#Read in the csvs
movie_df = pd.read_csv("Resources/movies.csv")
genre_df = pd.read_csv("Resources/movies_modified.csv")
df = pd.read_csv("Resources/average_ratings.csv")

#Retrieve the movieId
movie_id = movie_df.loc[movie_df["title"]=="Jumanji (1995)"].values[0][0]

#Retrieve the genres
genre_list = genre_df.loc[genre_df["movieId"]==movie_id].values[0][2:]

In [72]:
#Collect average ratings by genre per cluster
means = df.loc[df["clusters"]==cluster].groupby("clusters").mean()

#Calculate the average rating for the input movie
means = means.values[0][2:]

In [73]:
#Find the average rating by the cluster of a movie with each of these genres
weights_list = []
for i in range(len(genre_list)):
    weights_list.append(means[i]*genre_list[i])
output_average = sum(weights_list)/sum(genre_list)

#This is our output guess at how they would rate the movie
output_average

3.399744938867666